## **Setup**
Click the Play Button. No need to expand. Do not touch this. 
<details><summary>Packages to be downloaded</summary>
Things it needs to install:  <br>

 *    Verilator- the simulator  <br>
 *    PyGithub - Helps with simulation    <br>
 *    VCD - Generates waveforms from `.vcd` files  <br>
 *    Widgets - Each widget must first be generated then will be loaded when played
</details>

In [ ]:
#@title Install VCD
!git clone https://github.com/yne/vcd.git
!make -C /content/vcd
%cd /content/vcd
!make install
%cd /


In [2]:
#@title Import Files from Repo
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests

!mkdir -p /content/tmp_code
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = "https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/arithmetic_lab/files/%s" % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("import_all.py")
from import_all import *
import_source()
import_packages()
from simulation import *
from vcd2df import *

In [3]:
#@title Install Verilator
!apt-get install verilator >/dev/null

In [ ]:
#@title Install Wavedrom Extension
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git
%load_ext nb_js_diagrammers

## The Lab

### Full Adder

The first building block of your Circuit will be the full adder.  

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/arithmetic_lab/media/adder.png)

| Module Name: |Full_Add|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|a 	|Input 	|1| 	‘a’ operand input|
|b 	|Input 	|1 |	‘b’ operand input|
|cin |	Input| 	1| 	Carry in|
|s 	|Output 	|1| 	Sum output|
|co| 	Output 	|1| 	Carry out output|

It will add together `a` and `b` and a carry over from another module. (If it is the first bit the `Cin` will be `0`)   
It will then give `s` is the sum of the inputs. If there is a carry out, `co` should be `1`.


In [ ]:
#@title Launch Simulation Workspace 
createSimulationWorkSpace("tmp_code/full_add")


In [ ]:
#@title Verilator TestBench
# !verilator --cc full_add 
!verilator -Wall --trace -cc full_add.sv --exe tb_full_add.cpp
!make -C obj_dir -f Vfull_add.mk Vfull_add > /dev/null
!./obj_dir/Vfull_add

In [7]:
#@title Create WaveDrom
df2wd("full_add")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/full_add.txt
---

###What is a module?  



Instead of copying and pasting a Full-Adder over and over, we can instantiate it again as a module.  
This is the same concept as making a function call in python or C++. We are just reusing code we have already written.  
It follows the following format:  
`module_name module_title (.first_signal(first_connection), .second_signal(second_connection));`  
The first line will be whatever you have called your module. In this case it will be `Full_Add`.  
Then you must give each full adder a name. This part isn't important, but if its name has meaning it will be easier to understand your code. So, label each adder depending on its position.
Now you must define which internal signals go to which external signals.  
For example, if we want to add 8 bits together, for the first full adder, a should be the first bit of an a bit input. The `Cin` should be `0`. `s` should be the first bit of an output and `co` should be neither an input nor an output, but should instead be an intermediate signal that is used for the second bit.


###Creating an 8-bit adder


To create an 8-bit adder, you will need 8 one bit adders all linked together.

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/arithmetic_lab/media/add8.png)

| Module Name: |8Add|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|a 	|Input 	|8| 	‘a’ operand input|
|b 	|Input 	|8 |	‘b’ operand input|
|cin |	Input| 1	| 	Carry in|
|s 	|Output 	|8| 	Sum output|
|co| 	Output 	|1| 	Carry out output of the last bit|


You can use various internal signals for `Cin` and `Cout`.  
This module should have 8 instances of your Full_Add module.

In [ ]:
#@title Launch Simulation Workspace
createSimulationWorkSpace("tmp_code/add_8")

In [ ]:
#@title Verilator TestBench
!verilator --cc add_8 
!verilator -Wall --trace -cc add_8.sv --exe tb_add_8.cpp
!make -C obj_dir -f Vadd_8.mk Vadd_8 > /dev/null
!./obj_dir/Vadd_8

### Top level module

![picture](https://raw.githubusercontent.com/byuccl/digital_design_colab/master/Labs/arithmetic_lab/media/lab_top.png)

You probably noticed that there is no `led` or `sw` signal yet. This will be contained in a top level module. You will instantiate 8Add and feed into it all the necessary signals.

| Module Name: |arithemtic_top|||
| ----------- | ----------- |--|--|
| Port Name      | Direction       |Width|Function|
|sw 	|Input 	|16| Switches (`sw[15:8]` = `b`, `sw[7:0]` = `a` operand)	|
|btnc |	Input| 1	| Subtract when pressed|
|led	|Output 	|9| 	LEDs (`led[8]`= overflow, `led[7:0]` = sum)|  

While `btnc` is high, `b` should be inverted to subtract `b` from `a`.  
Led[8] should be high if overflow occurs.



### Overflow Detection  




###How to find of Overflow Occured

While using two's complement we can easily check for overflow. 
If the MSB's are the same, but the output is different then overflow has occurred. For example, `1111`+`1000`, both MSB are `1`. The output would be `0111`. Since `0`, the MSB, is different than `1`, we can tell that overflow has occurred.    


In other words, if you add two negative numbers and get a positive number, overflow has occurred. If you add two negative numbers and the output is positive, overflow has occurred.



###How does subtraction work?
As we explained earlier, there are two ways to do subtraction with two's complement, either using borrows like in elementary school, or inverting the second operator.  
Unfortunately, our computers never went to elementary school, so they will have to do the second option.  
Luckily, inverting `b` is quite simple.  
But you must also add `1` when inverting. Also, we have a carry in signal for our first bit, which acts exactly like adding `1`.    
This means that we can write our code such that if that carry in is high, then subtraction should occur.

In [ ]:
#@title Create Simluation Workspace
createSimulationWorkSpace("tmp_code/arithmetic_top")

In [13]:
#@title Generate an XDC File
%%bash -c 'cat > /content/tmp_code/xdc.xdc'
###################################################################
###################################################################
## Buttons
set_property -dict { PACKAGE_PIN U18   IOSTANDARD LVCMOS33 } [get_ports { btnc }];
# set_property -dict { PACKAGE_PIN T18   IOSTANDARD LVCMOS33 } [get_ports { btnu }];
# set_property -dict { PACKAGE_PIN W19   IOSTANDARD LVCMOS33 } [get_ports { btnl }];
# set_property -dict { PACKAGE_PIN T17   IOSTANDARD LVCMOS33 } [get_ports { btnr }];
# set_property -dict { PACKAGE_PIN U17   IOSTANDARD LVCMOS33 } [get_ports { btnd }];

## Switches
set_property -dict { PACKAGE_PIN V17   IOSTANDARD LVCMOS33 } [get_ports { sw[0] }];
set_property -dict { PACKAGE_PIN V16   IOSTANDARD LVCMOS33 } [get_ports { sw[1] }];
set_property -dict { PACKAGE_PIN W16   IOSTANDARD LVCMOS33 } [get_ports { sw[2] }];
set_property -dict { PACKAGE_PIN W17   IOSTANDARD LVCMOS33 } [get_ports { sw[3] }];
set_property -dict { PACKAGE_PIN W15   IOSTANDARD LVCMOS33 } [get_ports { sw[4] }];
set_property -dict { PACKAGE_PIN V15   IOSTANDARD LVCMOS33 } [get_ports { sw[5] }];
set_property -dict { PACKAGE_PIN W14   IOSTANDARD LVCMOS33 } [get_ports { sw[6] }];
set_property -dict { PACKAGE_PIN W13   IOSTANDARD LVCMOS33 } [get_ports { sw[7] }];
set_property -dict { PACKAGE_PIN V2    IOSTANDARD LVCMOS33 } [get_ports { sw[8] }];
set_property -dict { PACKAGE_PIN T3    IOSTANDARD LVCMOS33 } [get_ports { sw[9] }];
set_property -dict { PACKAGE_PIN T2    IOSTANDARD LVCMOS33 } [get_ports { sw[10] }];
set_property -dict { PACKAGE_PIN R3    IOSTANDARD LVCMOS33 } [get_ports { sw[11] }];
set_property -dict { PACKAGE_PIN W2    IOSTANDARD LVCMOS33 } [get_ports { sw[12] }];
set_property -dict { PACKAGE_PIN U1    IOSTANDARD LVCMOS33 } [get_ports { sw[13] }];
set_property -dict { PACKAGE_PIN T1    IOSTANDARD LVCMOS33 } [get_ports { sw[14] }];
set_property -dict { PACKAGE_PIN R2    IOSTANDARD LVCMOS33 } [get_ports { sw[15] }];

## LEDs
set_property -dict { PACKAGE_PIN U16   IOSTANDARD LVCMOS33 } [get_ports { led[0] }];
set_property -dict { PACKAGE_PIN E19   IOSTANDARD LVCMOS33 } [get_ports { led[1] }];
set_property -dict { PACKAGE_PIN U19   IOSTANDARD LVCMOS33 } [get_ports { led[2] }];
set_property -dict { PACKAGE_PIN V19   IOSTANDARD LVCMOS33 } [get_ports { led[3] }];
set_property -dict { PACKAGE_PIN W18   IOSTANDARD LVCMOS33 } [get_ports { led[4] }];
set_property -dict { PACKAGE_PIN U15   IOSTANDARD LVCMOS33 } [get_ports { led[5] }];
set_property -dict { PACKAGE_PIN U14   IOSTANDARD LVCMOS33 } [get_ports { led[6] }];
set_property -dict { PACKAGE_PIN V14   IOSTANDARD LVCMOS33 } [get_ports { led[7] }];
set_property -dict { PACKAGE_PIN V13   IOSTANDARD LVCMOS33 } [get_ports { led[8] }];
# set_property -dict { PACKAGE_PIN V3    IOSTANDARD LVCMOS33 } [get_ports { led[9] }];
# set_property -dict { PACKAGE_PIN W3    IOSTANDARD LVCMOS33 } [get_ports { led[10] }];
# set_property -dict { PACKAGE_PIN U3    IOSTANDARD LVCMOS33 } [get_ports { led[11] }];
# set_property -dict { PACKAGE_PIN P3    IOSTANDARD LVCMOS33 } [get_ports { led[12] }];
# set_property -dict { PACKAGE_PIN N3    IOSTANDARD LVCMOS33 } [get_ports { led[13] }];
# set_property -dict { PACKAGE_PIN P1    IOSTANDARD LVCMOS33 } [get_ports { led[14] }];
# set_property -dict { PACKAGE_PIN L1    IOSTANDARD LVCMOS33 } [get_ports { led[15] }];



## Compiling with the F4PGA Toolchain


### Installing the Toolchain

In [ ]:
#@title Install Dependencies
!apt install -y git wget xz-utils

In [ ]:
#@title Clone Repo
%%bash
cd /content
git clone https://github.com/chipsalliance/f4pga-examples
cd f4pga-examples
time wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O conda_installer.sh


In [ ]:
#@title Create the Conda Environment
%%bash
cd /content/f4pga-examples
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
time bash conda_installer.sh -u -b -p $F4PGA_INSTALL_DIR/$FPGA_FAM/conda;
time source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
time conda env create -f $FPGA_FAM/environment.yml

In [17]:
#@title Download F4PGA Arch Defs
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
conda activate xc7
mkdir -p $F4PGA_INSTALL_DIR/xc7/install

F4PGA_TIMESTAMP='20220907-210059'
F4PGA_HASH='66a976d'
export F4PGA_PACKAGES='install-xc7 xc7a50t_test'

for PKG in $F4PGA_PACKAGES; do
  wget -qO- https://storage.googleapis.com/symbiflow-arch-defs/artifacts/prod/foss-fpga-tools/symbiflow-arch-defs/continuous/install/${F4PGA_TIMESTAMP}/symbiflow-arch-defs-${PKG}-${F4PGA_HASH}.tar.xz | tar -xJC $F4PGA_INSTALL_DIR/${FPGA_FAM}
done

### Compiling with the Toolchain

In [21]:
#@title Create Makefile
with open("/content/tmp_code/Makefile", "w") as f:
  f.write("""current_dir := ${CURDIR}
TARGET := basys3

TOP := arithmetic_top

XDC := ${current_dir}/*.xdc

SOURCES := $(wildcard ${current_dir}/*.v ${current_dir}/*.sv)

include /content/f4pga-examples/common/common.mk
""")

In [22]:
#Make the project
%%bash
export F4PGA_INSTALL_DIR=~/opt/f4pga
export FPGA_FAM=xc7
export TARGET=basys3
export FOLDER=Arithmetic
export FILES="Makefile xdc.xdc arithmetic_top.sv full_add.sv add_8.sv"
source "$F4PGA_INSTALL_DIR/$FPGA_FAM/conda/etc/profile.d/conda.sh";
mkdir /content/"$FOLDER"
cd /content/tmp_code
for file in $FILES
do 
    mv $file /content/"$FOLDER"/$file
done
conda activate xc7
cd /content/$FOLDER
cp /content/tmp_code/errorFeedback.py ./
time python3 errorFeedback.py 2> ./error.log 1>./compile.log
if [ -f "/content/$FOLDER/build/basys3/*.bit" ]; then
    cp /content/$FOLDER/build/basys3/*.bit /content/$FOLDER.bit
else
    echo "An Error has occurred and the bit stream has not been generated:"
    cat ./compile.log
    cat ./error.log
fi

## Testing it on the board

You can use this configuration file and the program openOCD to download the bitstream to your board. You will need to download this file and the bitstream to your local machine.

In [ ]:
#@title Create File for openocd
folder = "Arithmetic"

with open(f"/content/{folder}.cfg", "w") as f:
  f.write("""interface ftdi
ftdi_device_desc "Digilent USB Device"
ftdi_vid_pid 0x0403 0x6010
# channel 1 does not have any functionality
ftdi_channel 0
# just TCK TDI TDO TMS, no reset
ftdi_layout_init 0x0088 0x008b
reset_config none
adapter_khz 10000

source [find cpld/xilinx-xc7.cfg]
source [find cpld/jtagspi.cfg]
init

puts [irscan xc7.tap 0x09]

set xc7a35t "0362D093"
set xc7a100t "13631093"
set code [drscan xc7.tap 32 0]  
puts $code

if { $code == $xc7a35t} {
    puts "The board has an xc7a35t"
}

if { $code == $xc7a100t} {
    puts "The board has an xc7a100t"
}

puts "Programming..."
"""
+
f"""
pld load 0 {folder}.bit
exit"""
)

The following code will download a zip file with the configuration file and bitstream. This will only download if you are using Chrome. If you are using something else, click the file icon on the left, and click the zip file in the content folder to download it.

In [ ]:
#@title Create Zip
%%bash
cd /content
export FOLDER=Arithmetic
cp $FOLDER/build/basys3/*.bit ./
zip $FOLDER.zip *.bit $FOLDER.cfg

Click [Here](https://colab.research.google.com/github/byuccl/digital_design_colab2/blob/master/Labs/seven_segment_lab/seven_segment_lab.ipynb) to move on to the Seven Segment Display Lab.